In [1]:
import pandas as pd
import numpy as np
import os
import collections
import pickle
import datetime
import re
from tqdm import trange
import argparse
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [2]:
from sklearn.preprocessing import LabelEncoder

In [3]:
with open('../LSTM/train_kr_all.pkl', 'rb') as f :
    train_kr = pickle.load(f)

In [4]:
train_kr['temp_code'] = train_kr.Script_num.map(lambda x : '-'.join(x.split('-')[:3]))

In [5]:
temp_code = 'UL-MV-01'
temp_script = []
temp_category = []
temp = ''
for i, row in train_kr.iterrows() :
    if temp_code == row.temp_code :
        temp += ' ' + ' '.join(row.Script)
    else :
        temp_code = row.temp_code
        temp_script.append(temp)
        temp_category.append(row.Category)
        temp = ''

In [6]:
temp_category = np.array(temp_category)

In [7]:
import collections

In [8]:
def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

In [123]:
vocabulary_size = 700

In [124]:
data, count, dictionary, reverse_dictionary = build_dataset(' '.join(temp_script).split(' '))

In [125]:
from sklearn.feature_extraction.text import CountVectorizer

In [126]:
count_vectorizer = CountVectorizer(vocabulary = list(dictionary.keys()))

In [127]:
raw_data = pd.DataFrame(count_vectorizer.fit_transform(temp_script).toarray(), columns = list(dictionary.keys()))

In [128]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [129]:
cv = KFold(n_splits = 5, shuffle = True, random_state = 7)

In [130]:
gnb = GaussianNB()

In [131]:
for train_index, test_index in cv.split(raw_data) :
    gnb.fit(raw_data.iloc[train_index], temp_category[train_index])
    yhat = gnb.predict(raw_data.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.41      0.43      0.42        47
         선박       0.38      0.45      0.41        53
         양묘       0.08      0.69      0.14        16
         육상       0.59      0.18      0.28       133
         이동       0.13      0.22      0.17        49
         입항       0.32      0.39      0.35        54
         접안       0.26      0.32      0.29        53
         출항       0.61      0.22      0.32        92
         통과       0.38      0.12      0.18       100
         투묘       0.32      0.30      0.31        40
         횡단       0.30      0.61      0.40        23

avg / total       0.41      0.28      0.29       660

             precision    recall  f1-score   support

         도선       0.33      0.46      0.38        35
         선박       0.35      0.29      0.32        55
         양묘       0.04      0.24      0.07        21
         육상       0.33      0.17      0.22       125
         이동       0.24      0.37      0.29

In [132]:
bnb = BernoulliNB()

In [133]:
for train_index, test_index in cv.split(raw_data) :
    bnb.fit(raw_data.iloc[train_index], temp_category[train_index])
    yhat = bnb.predict(raw_data.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.79      0.66      0.72        47
         선박       0.84      0.70      0.76        53
         양묘       0.60      0.19      0.29        16
         육상       0.68      0.79      0.73       133
         이동       0.61      0.71      0.66        49
         입항       0.81      0.70      0.75        54
         접안       0.67      0.85      0.75        53
         출항       0.90      0.80      0.85        92
         통과       0.71      0.74      0.73       100
         투묘       0.62      0.62      0.62        40
         횡단       0.70      0.61      0.65        23

avg / total       0.74      0.73      0.73       660

             precision    recall  f1-score   support

         도선       0.52      0.66      0.58        35
         선박       0.80      0.58      0.67        55
         양묘       0.60      0.14      0.23        21
         육상       0.59      0.66      0.62       125
         이동       0.68      0.72      0.70

In [134]:
mnb = MultinomialNB()

In [135]:
for train_index, test_index in cv.split(raw_data) :
    mnb.fit(raw_data.iloc[train_index], temp_category[train_index])
    yhat = mnb.predict(raw_data.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.80      0.74      0.77        47
         선박       0.87      0.74      0.80        53
         양묘       0.71      0.31      0.43        16
         육상       0.72      0.73      0.73       133
         이동       0.57      0.71      0.64        49
         입항       0.82      0.74      0.78        54
         접안       0.67      0.81      0.74        53
         출항       0.85      0.83      0.84        92
         통과       0.72      0.71      0.71       100
         투묘       0.63      0.68      0.65        40
         횡단       0.68      0.74      0.71        23

avg / total       0.74      0.73      0.73       660

             precision    recall  f1-score   support

         도선       0.61      0.66      0.63        35
         선박       0.76      0.58      0.66        55
         양묘       0.86      0.29      0.43        21
         육상       0.62      0.70      0.66       125
         이동       0.62      0.68      0.65

### Using tf-idf

In [136]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [137]:
tf = TfidfVectorizer(vocabulary = list(dictionary.keys()))

In [138]:
raw_data2 = pd.DataFrame(tf.fit_transform(temp_script).toarray(), columns = list(dictionary.keys()))

In [139]:
import xgboost

In [140]:
xgbc = xgboost.XGBClassifier(max_depth = 4, n_jobs = 12, n_estimators = 500)

In [141]:
for train_index, test_index in cv.split(raw_data) :
    xgbc.fit(raw_data.iloc[train_index], temp_category[train_index])
    yhat = xgbc.predict(raw_data.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.73      0.64      0.68        47
         선박       0.78      0.85      0.81        53
         양묘       0.57      0.75      0.65        16
         육상       0.72      0.70      0.71       133
         이동       0.76      0.71      0.74        49
         입항       0.80      0.74      0.77        54
         접안       0.87      0.89      0.88        53
         출항       0.81      0.85      0.83        92
         통과       0.80      0.73      0.76       100
         투묘       0.70      0.78      0.74        40
         횡단       0.62      0.78      0.69        23

avg / total       0.76      0.76      0.76       660

             precision    recall  f1-score   support

         도선       0.60      0.69      0.64        35
         선박       0.81      0.71      0.76        55
         양묘       0.75      0.57      0.65        21
         육상       0.62      0.73      0.67       125
         이동       0.75      0.74      0.74

In [142]:
for train_index, test_index in cv.split(raw_data2) :
    xgbc.fit(raw_data2.iloc[train_index], temp_category[train_index])
    yhat = xgbc.predict(raw_data2.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.79      0.64      0.71        47
         선박       0.72      0.83      0.77        53
         양묘       0.53      0.62      0.57        16
         육상       0.70      0.72      0.71       133
         이동       0.68      0.69      0.69        49
         입항       0.82      0.76      0.79        54
         접안       0.85      0.85      0.85        53
         출항       0.79      0.83      0.81        92
         통과       0.76      0.68      0.72       100
         투묘       0.74      0.72      0.73        40
         횡단       0.59      0.70      0.64        23

avg / total       0.74      0.74      0.74       660

             precision    recall  f1-score   support

         도선       0.69      0.77      0.73        35
         선박       0.82      0.76      0.79        55
         양묘       0.69      0.52      0.59        21
         육상       0.59      0.70      0.64       125
         이동       0.69      0.70      0.70

In [143]:
from sklearn.ensemble import RandomForestClassifier

In [144]:
rfc = RandomForestClassifier(max_depth = 15, n_jobs = 12, n_estimators = 1000)

In [145]:
for train_index, test_index in cv.split(raw_data2) :
    rfc.fit(raw_data2.iloc[train_index], temp_category[train_index])
    yhat = rfc.predict(raw_data2.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.76      0.55      0.64        47
         선박       0.93      0.53      0.67        53
         양묘       0.67      0.25      0.36        16
         육상       0.57      0.88      0.69       133
         이동       0.74      0.65      0.70        49
         입항       0.90      0.70      0.79        54
         접안       0.87      0.85      0.86        53
         출항       0.79      0.84      0.81        92
         통과       0.75      0.75      0.75       100
         투묘       0.79      0.57      0.67        40
         횡단       0.76      0.70      0.73        23

avg / total       0.76      0.73      0.73       660

             precision    recall  f1-score   support

         도선       0.72      0.60      0.66        35
         선박       0.83      0.36      0.51        55
         양묘       0.40      0.10      0.15        21
         육상       0.45      0.81      0.58       125
         이동       0.67      0.60      0.63

In [146]:
from sklearn.linear_model import LogisticRegression

In [147]:
lrc = LogisticRegression(C = 0.8, max_iter = 500, n_jobs = 12, solver = 'sag')

In [148]:
for train_index, test_index in cv.split(raw_data2) :
    lrc.fit(raw_data2.iloc[train_index], temp_category[train_index])
    yhat = lrc.predict(raw_data2.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.78      0.66      0.71        47
         선박       0.91      0.74      0.81        53
         양묘       0.75      0.56      0.64        16
         육상       0.66      0.83      0.74       133
         이동       0.75      0.67      0.71        49
         입항       0.87      0.76      0.81        54
         접안       0.91      0.81      0.86        53
         출항       0.80      0.84      0.82        92
         통과       0.72      0.75      0.74       100
         투묘       0.64      0.62      0.63        40
         횡단       0.74      0.61      0.67        23

avg / total       0.76      0.75      0.76       660

             precision    recall  f1-score   support

         도선       0.75      0.60      0.67        35
         선박       0.82      0.60      0.69        55
         양묘       0.64      0.33      0.44        21
         육상       0.53      0.79      0.63       125
         이동       0.67      0.67      0.67

In [149]:
import lightgbm

In [150]:
lgbmc = lightgbm.LGBMClassifier(n_jobs = 12, n_estimators = 500, max_depth = 5)

In [151]:
for train_index, test_index in cv.split(raw_data2) :
    lgbmc.fit(raw_data2.iloc[train_index].values, temp_category[train_index])
    yhat = lgbmc.predict(raw_data2.iloc[test_index].values)
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.76      0.68      0.72        47
         선박       0.67      0.83      0.74        53
         양묘       0.56      0.62      0.59        16
         육상       0.68      0.70      0.69       133
         이동       0.67      0.69      0.68        49
         입항       0.79      0.78      0.79        54
         접안       0.88      0.83      0.85        53
         출항       0.83      0.79      0.81        92
         통과       0.77      0.68      0.72       100
         투묘       0.64      0.70      0.67        40
         횡단       0.54      0.57      0.55        23

avg / total       0.73      0.73      0.73       660

             precision    recall  f1-score   support

         도선       0.67      0.69      0.68        35
         선박       0.82      0.76      0.79        55
         양묘       0.61      0.52      0.56        21
         육상       0.58      0.71      0.64       125
         이동       0.64      0.60      0.62

In [152]:
from sklearn.svm import LinearSVC

In [153]:
lsvc = LinearSVC(C = 0.7)

In [154]:
for train_index, test_index in cv.split(raw_data2) :
    lsvc.fit(raw_data2.iloc[train_index].values, temp_category[train_index])
    yhat = lsvc.predict(raw_data2.iloc[test_index].values)
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.78      0.66      0.71        47
         선박       0.81      0.79      0.80        53
         양묘       0.75      0.75      0.75        16
         육상       0.70      0.70      0.70       133
         이동       0.70      0.65      0.67        49
         입항       0.82      0.78      0.80        54
         접안       0.83      0.81      0.82        53
         출항       0.79      0.85      0.82        92
         통과       0.71      0.74      0.73       100
         투묘       0.69      0.72      0.71        40
         횡단       0.64      0.70      0.67        23

avg / total       0.75      0.75      0.75       660

             precision    recall  f1-score   support

         도선       0.75      0.77      0.76        35
         선박       0.76      0.69      0.72        55
         양묘       0.76      0.62      0.68        21
         육상       0.60      0.74      0.66       125
         이동       0.74      0.68      0.71

In [155]:
for train_index, test_index in cv.split(raw_data2) :
    mnb.fit(raw_data2.iloc[train_index], temp_category[train_index])
    yhat = mnb.predict(raw_data2.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.89      0.34      0.49        47
         선박       1.00      0.19      0.32        53
         양묘       0.00      0.00      0.00        16
         육상       0.53      0.91      0.67       133
         이동       0.62      0.53      0.57        49
         입항       0.86      0.46      0.60        54
         접안       0.89      0.79      0.84        53
         출항       0.59      0.86      0.70        92
         통과       0.63      0.72      0.67       100
         투묘       0.75      0.60      0.67        40
         횡단       1.00      0.13      0.23        23

avg / total       0.70      0.63      0.60       660

             precision    recall  f1-score   support

         도선       0.62      0.29      0.39        35
         선박       0.89      0.15      0.25        55
         양묘       0.00      0.00      0.00        21
         육상       0.46      0.86      0.60       125
         이동       0.64      0.53      0.58

/home/jin-lab/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Word Embedding으로 calssification 해보면?

In [156]:
from gensim.models import Word2Vec

In [157]:
embedding_model = Word2Vec.load('../LSTM/kr_word2vec_all.model')

In [158]:
def get_mean_vector(sentence) :
    words = sentence.split()
    result = []
    for word in words :
        try :
            result.append(embedding_model.wv.get_vector(word))
        except :
            continue
    result = np.array(result)
    if result.sum() == 0 :
        result = np.zeros((1, 300))
    result = result.mean(axis = 0)
    return result
    

In [159]:
embedding_data = np.array(None)
for sentence in temp_script :
    embedding_data = np.append(embedding_data, get_mean_vector(sentence))

In [160]:
embedding_data = np.delete(embedding_data, 0)

In [161]:
embedding_data = embedding_data.reshape(-1, 300)

In [190]:
xgbc2 = xgboost.XGBClassifier(max_depth = 4, n_jobs = 12, n_estimators = 500, random_state = 777)

In [191]:
for train_index, test_index in cv.split(embedding_data) :
    xgbc2.fit(embedding_data[train_index], temp_category[train_index])
    yhat = xgbc2.predict(embedding_data[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.73      0.68      0.70        47
         선박       0.71      0.66      0.69        53
         양묘       0.50      0.19      0.27        16
         육상       0.66      0.77      0.71       133
         이동       0.64      0.61      0.62        49
         입항       0.81      0.72      0.76        54
         접안       0.87      0.77      0.82        53
         출항       0.76      0.83      0.79        92
         통과       0.74      0.73      0.73       100
         투묘       0.59      0.65      0.62        40
         횡단       0.60      0.52      0.56        23

avg / total       0.71      0.71      0.71       660

             precision    recall  f1-score   support

         도선       0.60      0.77      0.68        35
         선박       0.81      0.62      0.70        55
         양묘       0.57      0.19      0.29        21
         육상       0.60      0.82      0.70       125
         이동       0.67      0.60      0.63

In [192]:
# lrc2 = LogisticRegression(C = 0.8, max_iter = 500, n_jobs = 12, solver = 'lbfgs')

In [193]:
# for train_index, test_index in cv.split(embedding_data) :
#     lrc2.fit(embedding_data[train_index], temp_category[train_index])
#     yhat = lrc2.predict(embedding_data[test_index])
#     print(classification_report(temp_category[test_index], yhat))

logistic regression은 성능이 너무 안나와서 코드 실행 x

### N-gram 사용

In [223]:
with open('../LSTM/data_for_classification.pickle', 'rb') as f :
    raw_data = pickle.load(f)

In [225]:
raw_data['temp_code'] = raw_data.Script_num.map(lambda x : '-'.join(x.split('-')[:3]))

In [226]:
raw_data.head()

,Script_num,Script,Language,Category,temp_code
0,UL-MV-01-02,아 울산VTS [SN] 감도 있습니까?,KR,이동,UL-MV-01
1,UL-MV-01-03,네 [SN],KR,이동,UL-MV-01
2,UL-MV-01-04,네 수고 많으십니다 본선 델타라인 통과해서 본선 1부두 접안하러 이동하겠습니다,KR,이동,UL-MV-01
3,UL-MV-01-05,들어오세요,KR,이동,UL-MV-01
4,UL-MV-01-06,네 저희 지금…,KR,이동,UL-MV-01


In [231]:
temp_code2 = 'UL-MV-01'
temp_script2 = []
temp_category2 = []
temp2 = ''
for i, row in raw_data.iterrows() :
    if temp_code2 == row.temp_code :
        temp2 += ' ' + ''.join(row.Script)
    else :
        temp_code2 = row.temp_code
        temp_script2.append(temp2.strip())
        temp_category2.append(row.Category)
        temp2 = ''

In [232]:
temp_script2

['아  울산VTS [SN] 감도 있습니까?  네  [SN]  네  수고 많으십니다 본선  델타라인  통과해서 본선  1부두  접안하러 이동하겠습니다  들어오세요  네 저희  지금…',
 '항무울산 [SN] 감도 있습니까?  네 [SN] [SN]  네  수고 많으십니다 본선  콜사인 DSN... 저희 본선 접안 스케줄이 변경되서 5번으로 투묘 차 이동하겠습니다 E1 VHF 하나만 부탁드리겠습니다  네 귀선  몇 톤입니까?  네 총 톤수 30901입니다 그리고 저희 앞에 배가 안 떨어져서 한 5시 경에 접안 차  이동할 예정입니다  네 화암추 등대로부터 156도 1.75마일  가십시오',
 '항무울산 [SN] 감도 있습니까? 오버  네,  [SN]  네,  수고 많으십니다 E 묘지에서 작업 종료 후 존생보안으로 이동하겠습니다 오버  수고하십시오',
 '항무울산 [SN]입니다  네,  [SN]  네,  이동입니다 10분 후에 중공업 1함대에서 장생포로  가고 971160입니다  네,  [SN],  수고하세요',
 '항무울산 [SN]  [SN],  항무울산  네,  수고 많으십니다 공사 현장에서 정일물항장 좀 가겠습니다 우선 이안해놓고  조심해서 이동하십시오',
 '항무울산 [SN]입니다  [SN],  항무울산  네,  [SN]. 이동 중입니다 마이크 3에서 SK 2-6으로  갑니다 어...  이동 10분 전입니다 선박번호 071242 이상입니다  <unintelligible>***</unintelligible>하십시오  네,  양지했습니다',
 '어,  항무울산 [SN] 감도 있습니까?  [SN],  항무울산  네,  수고하십니다 이동 예고입니다 본선  앞으로 10분 후 양묘하여 본항 4-2번으로 이동합니다 콜사인 DSND9 이상입니다  [SN] 지금  본항 4-2번에 배가 붙어있는데요?  <unintelligible>** ***</untelligible>o ship-to-ship <unintelligible>** ***</untelligible> 입니다.  네,

In [195]:
from sklearn.feature_extraction.text import CountVectorizer

In [216]:
ngram_vectorizer = CountVectorizer(ngram_range = (2, 3), )

In [217]:
ngram = ngram_vectorizer.fit_transform(temp_script)

In [218]:
ngram = ngram.toarray()

In [220]:
ngram[0]

(3300, 54592)

### Time information 반영

In [ ]:
temp_code = 'UL-MV-01'
temp_script2 = []
temp_category2 = []
temp2 = ''
step = []
temp_step = 0
for i, row in train_kr.iterrows() :
    temp_step += 1
    if temp_code == row.temp_code :
        temp2 += ' ' + ' '.join(row.Script)
        temp_script2.append(temp2)
        temp_category2.append(row.Category)
        step.append(temp_step)
    else :
        temp_code = row.temp_code
        temp_script2.append(temp2)
        temp_category2.append(row.Category)
        temp2 = ''
        step.append(temp_step)
        temp_step = 0

In [ ]:
temp_df = pd.DataFrame(tf.transform(temp_script2).toarray(), columns = list(dictionary.keys()))

In [ ]:
temp_df['category'] = temp_category2
temp_df['step'] = step

상위 3개 예측 category에 실제 label이 들어가면 정답

In [ ]:
for i in range(1, 25) :
    data = temp_df[temp_df.step == i].drop(['category', 'step'], axis = 1)
    yhat = xgbc.predict(data)
    print(classification_report(temp_df[temp_df.step == i].category.values, yhat), i)